# 밑준비

## 라이브러리 불러오기

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer, tokenizer_from_json
from tensorflow.keras.utils import pad_sequences, to_categorical
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec, FastText

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

import pandas as pd
import numpy as np

from copy import deepcopy
from collections import Counter
import pytz, re, json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 전처리 옵션

In [ ]:
# embedding_model = 'w2v'
embedding_model = 'fast'
pattern = r'[\s\-_\(\)\/]'

alarmno_len = 12
alarmlv_len = 12
alarmmsg_len = 12
site_len = 12
sys_len = 12

full_name =  {
'ais': 'alarm indication signal',
'batt': 'battery',
'cep': 'critical event prevention',
'conn': 'connection',
'crc': 'cyclic redundancy check',
'csf': 'client signal fail',
'dcc': 'data communication channel',
'dcn': 'data communication network',
'dc': 'direct current',
'ddm': 'digital diagnostic monitoring',
'env': 'environment',
'err': 'error',
'eth': 'ethernet',
'fan': 'fan',
'fl': 'frame loss',
'idf': 'in-service data frame',
'init': 'initialization',
'ioc': 'input output controller',
'ipc': 'inter process communication',
'iof': 'ingress only filtering',
'll': 'link-layer',
'llcf': 'link level control field',
'loc': 'loss of communication',
'loc': 'loss of continuity',
'lof': 'loss of frame',
'los': 'loss of signal',
'lsp': 'label switched path',
'meg': 'mega',
'mep': 'maintenance entity group end point',
'mis': 'misalignment',
'ne': 'network element',
'nvram': 'non volatile random access memory',
'oam': 'operations, administration, and maintenance',
'oamloss': 'operations administration and maintenance loss',
'odi': 'optical delay interferometer',
'opt': 'optical',
'pde': 'path delay estimation',
'pdm': 'physical data model',
'poam': 'physical operations, administration, and maintenance',
'ps': 'packet switching',
'psm': 'power save mode',
'psu': 'power supply unit',
'pw': 'pseudowire',
'pwe': 'pseudowire emulation',
'ql': 'queue length',
'rdi': 'remote defect indication',
'rem': 'remote',
'rfa': 'remote failure analysis',
'rmt': 'remote',
'rx': 'receiver',
'rsmt': 'required signal-to-noise ratio margin threshold',
'sfp': 'small form factor pluggable',
'stm': 'synchronous transport module',
'sys': 'system',
'tca': 'threshold crossing alert',
'tdm': 'time division multiplexing',
'trk': 'track',
'tx': 'transmit',
'urc': 'unidirectional remote control',
}

# 데이터 불러오기

In [ ]:
train_path = '/content/drive/MyDrive/my_data/kt_network/q2/Q2_train.csv'
test_path = '/content/drive/MyDrive/my_data/kt_network/q2/Q2_test.csv'
submisson_path = '/content/drive/MyDrive/my_data/kt_network/q2/Q2_label_sample.csv'
save_path = '/content/drive/MyDrive/kt_network_competition/AutogluonModels_kt_network'
w2v_path = '/content/drive/MyDrive/kt_network_competition/w2v_model'
fast_path = '/content/drive/MyDrive/kt_network_competition/fasttext_model'
tk_path = '/content/drive/MyDrive/kt_network_competition/'
rnn_path = '/content/drive/MyDrive/my_data/kt_network/model/q2model.h5'

In [ ]:
target = "root_cause_type"

data = pd.read_csv('/content/drive/MyDrive/my_data/kt_network/q2/Q2_train.csv')
data

,ticketno,alarmno,alarmtime,alarmlevel,alarmmsg_original,site,sysname,unit,slot,port,sva,root_cause_domain,root_cause_type
0,21122633.0,1669820428245,2022-12-01 00:02:24+09:00,5,ETH-ERR,ACEN,acnt,EQPT,3.0,1.0,NSA,A,LinkCut
1,21122633.0,1669821318728,2022-12-01 00:17:15+09:00,5,ETH-ERR,ACEN,acnt,EQPT,3.0,1.0,NSA,A,LinkCut
2,21122633.0,1669822214832,2022-12-01 00:32:11+09:00,5,ETH-ERR,ACEN,acnt,EQPT,3.0,1.0,NSA,A,LinkCut
3,21122633.0,1669823114128,2022-12-01 00:47:10+09:00,5,ETH-ERR,ACEN,acnt,EQPT,3.0,1.0,NSA,A,LinkCut
4,21122633.0,1669824028082,2022-12-01 01:02:24+09:00,5,ETH-ERR,ACEN,acnt,EQPT,3.0,1.0,NSA,A,LinkCut
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9317,21774618.0,1671892499215,2022-12-24 23:37:14+09:00,7,OPT-LOS,AFAA,agow,EQPT,5.0,5.0,NSA,A,LinkCut
9318,15693425.0,877949375,2022-12-25 10:13:46+09:00,7,OPT-REMOVE,AGFD,aibb,EQPT,1.0,8.0,NSA,A,UnitFail
9319,21809789.0,1671974758375,2022-12-25 22:28:14+09:00,7,OPT-LOS,ADKA,aeaq,EQPT,3.0,3.0,NSA,A,LinkCut
9320,21811213.0,1671978167736,2022-12-25 23:25:03+09:00,7,OPT-LOS,ABZO,acie,EQPT,5.0,6.0,NSA,A,LinkCut


In [ ]:
# 모델 생성
if embedding_model == 'w2v':
    model = Word2Vec.load(w2v_path)
elif embedding_model == 'fast':
    model = FastText.load(fast_path)
# 단어를 임베딩한 벡터 값 확인
print(model.wv['communication', 'data', 'network'])

[[ 2.33462139e-04  1.10550527e-03 -8.30097066e-04 -9.65214160e-04
   1.65458478e-03  1.84627905e-04  1.73240493e-03 -2.34158098e-04
  -2.08324310e-03 -1.01679834e-04  2.85268109e-03  7.06844672e-04
   1.54870690e-03  2.28503501e-04  9.95156239e-04  9.44359053e-04
  -1.14503561e-03  4.91483370e-04 -9.21080296e-04  1.61688324e-04
   6.10307325e-04  6.01704989e-04  3.35507822e-04 -9.15470295e-08
  -4.91310318e-04 -8.73949553e-04 -1.35218864e-03  1.55750095e-04
  -1.55129033e-04 -8.36733088e-05  1.01745839e-03  7.60567491e-04
  -2.55621108e-03 -6.10819203e-04  4.00126482e-05 -1.01661379e-03
   2.64662970e-03  1.94886758e-04 -5.50074619e-04  6.32610230e-04
  -9.65014915e-04 -8.58902989e-04 -1.65244425e-03  1.20835635e-03
   1.42975070e-03 -1.72063825e-03  7.63372867e-04 -9.44979955e-04
  -5.51469275e-05 -9.46307147e-04  2.99201114e-03 -7.33090856e-04
  -1.73058652e-04 -7.49957835e-05  2.86441045e-05 -2.30262452e-03
  -3.40513740e-04  5.78511972e-04  1.29883550e-03  2.06581061e-03
   1.45014

# 전처리

In [ ]:
# 축약어를 문장화 한 뒤 토큰화
def expand_abbr(abbr_lst, abbr_dict):
    temp = []
    for abbr in abbr_lst:
        if abbr in abbr_dict:
            temp.extend(abbr_dict[abbr].split())
        else:
            if len(abbr) > 1:
                temp.append(abbr)
    return temp

# 벡터들의 평균 구하기
def mean_vector(vectors):
    return np.mean(vectors, axis=0)


def feature_engineering(data, pattern, abbr_dict):
    '''
    params
        data : DataFrame - feature engineering의 대상이 되는 데이터
    return
        DataFrame
    '''
    # ticketno column을 int64로 변경
    temp = deepcopy(data)
    temp['ticketno'] = temp['ticketno'].astype('int64')

    # ticketno, alarmno를 로그변환
    temp['ticketno_log1p'] = np.log1p(temp['ticketno'])
    temp['alarmno_log1p'] = np.log1p(temp['alarmno'])

    # alarmtime column을 datetime64로 변경
    tz_pytz = pytz.timezone('Asia/Seoul')
    temp['alarmtime'] = pd.to_datetime(temp['alarmtime'], unit='ns', utc=True).dt.tz_convert('Asia/Seoul')

    # alarmmsg
    temp["alarmmsg_vector"] = temp["alarmmsg_original"].str.lower()
        # 불용어 처리
    temp['alarmmsg_vector'] = temp['alarmmsg_vector'].apply(lambda x : re.sub(pattern, ' ', x))
        # 토큰화
    temp['alarmmsg_vector'] = temp['alarmmsg_vector'].apply(lambda x : re.findall('[A-Za-z0-9]+', x))
        # 축약어 풀이 및 토큰화
    temp['alarmmsg_vector'] = temp['alarmmsg_vector'].apply(lambda x : expand_abbr(x, abbr_dict))
        # 토큰 임베딩
    temp['alarmmsg_vector'] = temp['alarmmsg_vector'].apply(lambda x : model.wv[x])
        # 수치표현된 토큰 값들을 평균내어 alarmmsg_original에 대한 수치표현 값 구하기
    temp['alarmmsg_vector'] = temp['alarmmsg_vector'].apply(lambda x : mean_vector(x))
        # 벡터에 차원을 추가해 다차원 벡터값들을 합치기 편하게 만든다.
    temp['alarmmsg_vector'] = temp['alarmmsg_vector'].apply(lambda x : np.expand_dims(x, axis=0))


    # alarmlevel 범주화
    temp['alarmlevel'] = temp['alarmlevel'].astype('object')

    # site & sysname을 알파벳 단위로 분리
    # site
    site_col_dict = {}
    for i in range(4):
        site_col_dict[f'site{i}'] = []

    for idx, value in temp['site'].items():
        for num, each_char in enumerate(value):
            site_col_dict[f'site{num}'].append(each_char)
    site_df = pd.DataFrame(site_col_dict)
    # sysname
    sysname_col_dict = {}
    for i in range(4):
        sysname_col_dict[f'sysname{i}'] = []

    for idx, value in temp['sysname'].items():
        for num, each_char in enumerate(value):
            sysname_col_dict[f'sysname{num}'].append(each_char)

    sysname_df = pd.DataFrame(sysname_col_dict)
    temp = pd.concat([temp, site_df, sysname_df], axis=1)

    return temp

## sequence 데이터화

In [ ]:
def make_seq_data(data, train=True):

    temp = deepcopy(data)

    # seq 데이터 생성
    temp_seq = temp.groupby(['ticketno'], group_keys=True, as_index=False)['alarmno_log1p'].apply(lambda x : ' '.join(map(str, x)))
        # ticketno 별로 그룹화 해서 sequence 데이터화 시키기
    temp_seq['ticketno_log1p'] = temp.groupby(['ticketno'], group_keys=True, as_index=False)['ticketno_log1p'].apply(lambda x : ' '.join(map(str, x)))['ticketno_log1p']
    temp_seq['alarmlevel'] = temp.groupby(['ticketno'], group_keys=True, as_index=False)['alarmlevel'].apply(lambda x : ' '.join(map(str, x)))['alarmlevel']
    temp_seq['alarmmsg_vector'] = temp.groupby(['ticketno'], group_keys=True, as_index=False)['alarmmsg_vector'].apply(lambda x: np.array(x)).apply(lambda x : np.concatenate(x, axis=0))
    temp_seq['site1'] = temp.groupby(['ticketno'], group_keys=True, as_index=False)['site1'].apply(lambda x : ' '.join(x))['site1']
    temp_seq['sysname1'] = temp.groupby(['ticketno'], group_keys=True, as_index=False)['sysname1'].apply(lambda x : ' '.join(x))['sysname1']


    # seq에서 새로운 정보 추출 및 데이터형 변환
    temp_seq['alarmno_log1p'] = temp_seq['alarmno_log1p'].apply(lambda x: list(map(float, x.split(' '))))
    temp_seq['ticketno_log1p'] = temp_seq['ticketno_log1p'].apply(lambda x: list(map(float, x.split(' '))))
    temp_seq['timesteps'] = temp_seq['alarmlevel'].apply(lambda x: len(x.split(' ')))


    if train == True:
    # 라벨링
        # 데이터 프레임의 feature중 list 또는 ndarray를 값으로 가지고 있는 feature가 있다면 drop_duplicate 실행시 오류가 난다.
        # 중복 제거 기준을 축소시킴으로서 해결하자.
        temp_seq = pd.merge(temp_seq, temp[['ticketno', target]], how='left', on='ticketno').drop_duplicates(subset='ticketno', keep='first', ignore_index=True)
    else:
        pass

    return temp_seq

In [ ]:
# alarmmsg_original을 다차원 벡터로 변환시키는 것은 autogluon 모델링과 동일하다.
temp = feature_engineering(data, pattern, full_name)
temp

,ticketno,alarmno,alarmtime,alarmlevel,alarmmsg_original,site,sysname,unit,slot,port,...,alarmno_log1p,alarmmsg_vector,site0,site1,site2,site3,sysname0,sysname1,sysname2,sysname3
0,21122633,1669820428245,2022-12-01 00:02:24+09:00,5,ETH-ERR,ACEN,acnt,EQPT,3.0,1.0,...,28.143737,"[[-0.0009134301, -0.00068933005, -0.0003933129...",A,C,E,N,a,c,n,t
1,21122633,1669821318728,2022-12-01 00:17:15+09:00,5,ETH-ERR,ACEN,acnt,EQPT,3.0,1.0,...,28.143738,"[[-0.0009134301, -0.00068933005, -0.0003933129...",A,C,E,N,a,c,n,t
2,21122633,1669822214832,2022-12-01 00:32:11+09:00,5,ETH-ERR,ACEN,acnt,EQPT,3.0,1.0,...,28.143738,"[[-0.0009134301, -0.00068933005, -0.0003933129...",A,C,E,N,a,c,n,t
3,21122633,1669823114128,2022-12-01 00:47:10+09:00,5,ETH-ERR,ACEN,acnt,EQPT,3.0,1.0,...,28.143739,"[[-0.0009134301, -0.00068933005, -0.0003933129...",A,C,E,N,a,c,n,t
4,21122633,1669824028082,2022-12-01 01:02:24+09:00,5,ETH-ERR,ACEN,acnt,EQPT,3.0,1.0,...,28.143739,"[[-0.0009134301, -0.00068933005, -0.0003933129...",A,C,E,N,a,c,n,t
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9317,21774618,1671892499215,2022-12-24 23:37:14+09:00,7,OPT-LOS,AFAA,agow,EQPT,5.0,5.0,...,28.144977,"[[-0.0008455632, 0.0013156387, 0.0017750752, 0...",A,F,A,A,a,g,o,w
9318,15693425,877949375,2022-12-25 10:13:46+09:00,7,OPT-REMOVE,AGFD,aibb,EQPT,1.0,8.0,...,20.593099,"[[-0.0020802815, 0.0030788938, 0.0017263817, 0...",A,G,F,D,a,i,b,b
9319,21809789,1671974758375,2022-12-25 22:28:14+09:00,7,OPT-LOS,ADKA,aeaq,EQPT,3.0,3.0,...,28.145027,"[[-0.0008455632, 0.0013156387, 0.0017750752, 0...",A,D,K,A,a,e,a,q
9320,21811213,1671978167736,2022-12-25 23:25:03+09:00,7,OPT-LOS,ABZO,acie,EQPT,5.0,6.0,...,28.145029,"[[-0.0008455632, 0.0013156387, 0.0017750752, 0...",A,B,Z,O,a,c,i,e


In [ ]:
# 이후 target에 대한 one-hot encoding을 위해 미리 mapping
label_dict = {'LinkCut' : 0, 'PowerFail' : 1, 'UnitFail' : 2}
temp['root_cause_type'] = temp['root_cause_type'].map(label_dict)
temp['root_cause_type']

0       0
1       0
2       0
3       0
4       0
       ..
9317    0
9318    2
9319    0
9320    0
9321    0
Name: root_cause_type, Length: 9322, dtype: int64

In [ ]:
# sequence 데이터 화
temp_seq = make_seq_data(temp, train=True)
temp_seq

,ticketno,alarmno_log1p,ticketno_log1p,alarmlevel,alarmmsg_vector,site1,sysname1,timesteps,root_cause_type
0,14753084,"[20.581846324460642, 20.581846326764456, 20.58...","[16.506962771423133, 16.506962771423133, 16.50...",4 7 4 7 7 5,"[[0.00034126244, -0.000639107, 0.00067938084, ...",B B B B B B,b b b b b b,6,1
1,14771766,"[20.582054895929588, 20.582054894777922, 20.58...","[16.508228281745982, 16.508228281745982, 16.50...",7 4 4 7,"[[0.0009238396, -0.00025987678, 0.00069495384,...",B B B B,b b b b,4,1
2,14777089,"[20.58210410432185, 20.582104102018626, 20.582...","[16.508588566413344, 16.508588566413344, 16.50...",4 4 7 7 7,"[[0.00034126244, -0.000639107, 0.00067938084, ...",B B B B B,b b b b b,5,1
3,14790052,"[20.582242987011327, 20.58224298816278]","[16.509465418182927, 16.509465418182927]",4 4,"[[0.00034126244, -0.000639107, 0.00067938084, ...",C C,d d,2,1
4,14879922,"[20.583055085271425, 20.58305508642194, 20.583...","[16.51552341262465, 16.51552341262465, 16.5155...",7 7 4 4 7 5,"[[0.0009238396, -0.00025987678, 0.00069495384,...",B B B B B B,b b b b b b,6,1
...,...,...,...,...,...,...,...,...,...
1109,21792877,[28.144951202142483],[16.89709377713444],7,"[[-0.0008455632, 0.0013156387, 0.0017750752, 0...",B,c,1,0
1110,21793984,"[28.144952713177496, 28.14496006365392]","[16.897144572255907, 16.897144572255907]",7 7,"[[-0.0008455632, 0.0013156387, 0.0017750752, 0...",D D,e e,2,0
1111,21799077,"[28.144959379526117, 28.14495937944836, 28.144...","[16.897378233286936, 16.897378233286936, 16.89...",7 7 5 5 5 5,"[[-0.0008455632, 0.0013156387, 0.0017750752, 0...",B B B B B B,c c c c c c,6,0
1112,21809789,"[28.1449748391285, 28.14502653381203]","[16.897869509520728, 16.897869509520728]",7 7,"[[-0.0008455632, 0.0013156387, 0.0017750752, 0...",D D,e e,2,0


## Train Test split

In [ ]:
x, y = temp_seq.drop(columns=[target]), temp_seq[target]
x.shape, y.shape

((1114, 8), (1114,))

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(x, y, random_state=2023, test_size=0.2, stratify=y)
x_train.shape, x_valid.shape, y_train.shape, y_valid.shape

((891, 8), (223, 8), (891,), (223,))

In [ ]:
train_seq = pd.concat([x_train, y_train], axis=1)
valid_seq = pd.concat([x_valid, y_valid], axis=1)
train_seq.shape, valid_seq.shape

((891, 9), (223, 9))

In [ ]:
train_seq

,ticketno,alarmno_log1p,ticketno_log1p,alarmlevel,alarmmsg_vector,site1,sysname1,timesteps,root_cause_type
356,18383456,[20.63926354303098],[16.726961742080938],7,"[[-0.0020802815, 0.0030788938, 0.0017263817, 0...",D,e,1,2
315,18380373,[20.63917973534545],[16.726794022889365],7,"[[-0.0020802815, 0.0030788938, 0.0017263817, 0...",E,f,1,2
883,21604385,"[28.14467274888314, 28.144672749446148, 28.144...","[16.88840690759696, 16.88840690759696, 16.8884...",5 5 5 5,"[[-0.0010747955, 0.00041649968, 0.00029036976,...",F F F F,h h h h,4,0
747,21401465,"[28.144332643661315, 28.14433300316006]","[16.87896998231864, 16.87896998231864]",7 7,"[[-0.0008455632, 0.0013156387, 0.0017750752, 0...",E E,f f,2,0
169,17715343,"[20.626056248707442, 20.626056259728355]","[16.689941714665004, 16.689941714665004]",7 4,"[[0.0009238396, -0.00025987678, 0.00069495384,...",C C,c c,2,1
...,...,...,...,...,...,...,...,...,...
894,21605550,[28.14467452146415],[16.88846083037867],5,"[[-0.0010747955, 0.00041649968, 0.00029036976,...",B,c,1,0
313,18380359,[20.639179518888643],[16.726793261207074],7,"[[-0.0020802815, 0.0030788938, 0.0017263817, 0...",G,i,1,2
23,15367349,[20.588888710790084],[16.547755686856707],7,"[[-0.0020802815, 0.0030788938, 0.0017263817, 0...",C,d,1,2
327,18381440,"[20.639209952908903, 20.639210341214042]","[16.726852072253916, 16.726852072253916]",7 7,"[[-0.0020802815, 0.0030788938, 0.0017263817, 0...",F F,h h,2,2


## alarmno seq

In [ ]:
# 이미 수치화 되어 있으므로 padding 및 차원 추가
train_alarmno_padseq = pad_sequences(list(train_seq['alarmno_log1p']), maxlen=alarmno_len, dtype='float', padding='pre', truncating='post')
valid_alarmno_padseq = pad_sequences(list(valid_seq['alarmno_log1p']), maxlen=alarmno_len, dtype='float', padding='pre', truncating='post')
train_alarmno_padseq = np.expand_dims(train_alarmno_padseq, axis=-1)
valid_alarmno_padseq = np.expand_dims(valid_alarmno_padseq, axis=-1)
train_alarmno_padseq[0]

array([[ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [20.63926354]])

## alarmlevel seq

In [ ]:
# alarmlevel을 범주형 데이터로 취급하기로 했으므로,
# 각 level을 토큰화
alarmlv_tokenizer = Tokenizer(num_words=4, filters='', split=' ')
alarmlv_tokenizer.fit_on_texts(temp_seq['alarmlevel'])
alarmlv_tokenizer.get_config()

{'num_words': 4,
 'filters': '',
 'lower': True,
 'split': ' ',
 'char_level': False,
 'oov_token': None,
 'document_count': 1114,
 'word_counts': '{"4": 375, "7": 2011, "5": 6922, "3": 14}',
 'word_docs': '{"7": 875, "5": 308, "4": 210, "3": 6}',
 'index_docs': '{"2": 875, "1": 308, "3": 210, "4": 6}',
 'index_word': '{"1": "5", "2": "7", "3": "4", "4": "3"}',
 'word_index': '{"5": 1, "7": 2, "4": 3, "3": 4}'}

In [ ]:
alarmlevel_vocab_size = alarmlv_tokenizer.num_words + 1
alarmlevel_vocab_size

5

In [ ]:
# 토큰화된 level을 idx sequence화
train_alarmlv_idxseq = alarmlv_tokenizer.texts_to_sequences(train_seq['alarmlevel'])
valid_alarmlv_idxseq = alarmlv_tokenizer.texts_to_sequences(valid_seq['alarmlevel'])

In [ ]:
# padding
train_alarmlv_padseq = pad_sequences(train_alarmlv_idxseq, maxlen=alarmlv_len, padding='pre', truncating='post')
valid_alarmlv_padseq = pad_sequences(valid_alarmlv_idxseq, maxlen=alarmlv_len, padding='pre', truncating='post')
train_alarmlv_padseq[:5]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 3]], dtype=int32)

## alarmmsg_original seq

In [ ]:
# 이미 수치 표현되어 있으므로 padding 만 해준다.
train_msgvec_padseq = pad_sequences(train_seq['alarmmsg_vector'], maxlen=alarmmsg_len, dtype='float', padding='pre', truncating='post')
valid_msgvec_padseq = pad_sequences(valid_seq['alarmmsg_vector'], maxlen=alarmmsg_len, dtype='float', padding='pre', truncating='post')

## site&sysname seq

In [ ]:
# alarmlevel의 경우와 같다.
site1_tokenizer = Tokenizer(num_words=6, filters='', split=' ', oov_token='<oov>')
sysname1_tokenizer = Tokenizer(num_words=6 , filters='', split=' ', oov_token='<oov>')

site1_tokenizer.fit_on_texts(temp['site1'])
sysname1_tokenizer.fit_on_texts(temp['sysname1'])

train_site1_idxseq = site1_tokenizer.texts_to_sequences(train_seq['site1'])
train_sysname1_idxseq = sysname1_tokenizer.texts_to_sequences(train_seq['sysname1'])
valid_site1_idxseq = site1_tokenizer.texts_to_sequences(valid_seq['site1'])
valid_sysname1_idxseq = sysname1_tokenizer.texts_to_sequences(valid_seq['sysname1'])

train_site1_padseq = pad_sequences(train_site1_idxseq, maxlen=site_len, padding='pre', truncating='post')
train_sysname1_padseq = pad_sequences(train_sysname1_idxseq, maxlen=site_len, padding='pre', truncating='post')
valid_site1_padseq = pad_sequences(valid_site1_idxseq, maxlen=sys_len, padding='pre', truncating='post')
valid_sysname1_padseq = pad_sequences(valid_sysname1_idxseq, maxlen=sys_len, padding='pre', truncating='post')

In [ ]:
site1_vocab_size = site1_tokenizer.num_words + 1
sysname1_vocab_size = sysname1_tokenizer.num_words + 1
site1_vocab_size, sysname1_vocab_size

(7, 7)

In [ ]:
site1_tokenizer.get_config()

{'num_words': 6,
 'filters': '',
 'lower': True,
 'split': ' ',
 'char_level': False,
 'oov_token': '<oov>',
 'document_count': 9322,
 'word_counts': '{"c": 1678, "e": 4872, "b": 650, "d": 1039, "f": 1017, "a": 34, "g": 32}',
 'word_docs': '{"c": 1678, "e": 4872, "b": 650, "d": 1039, "f": 1017, "a": 34, "g": 32}',
 'index_docs': '{"3": 1678, "2": 4872, "6": 650, "4": 1039, "5": 1017, "7": 34, "8": 32}',
 'index_word': '{"1": "<oov>", "2": "e", "3": "c", "4": "d", "5": "f", "6": "b", "7": "a", "8": "g"}',
 'word_index': '{"<oov>": 1, "e": 2, "c": 3, "d": 4, "f": 5, "b": 6, "a": 7, "g": 8}'}

In [ ]:
sysname1_tokenizer.get_config()

{'num_words': 6,
 'filters': '',
 'lower': True,
 'split': ' ',
 'char_level': False,
 'oov_token': '<oov>',
 'document_count': 9322,
 'word_counts': '{"c": 2176, "f": 4678, "e": 993, "b": 98, "g": 417, "h": 802, "d": 98, "a": 36, "i": 24}',
 'word_docs': '{"c": 2176, "f": 4678, "e": 993, "b": 98, "g": 417, "h": 802, "d": 98, "a": 36, "i": 24}',
 'index_docs': '{"3": 2176, "2": 4678, "4": 993, "7": 98, "6": 417, "5": 802, "8": 98, "9": 36, "10": 24}',
 'index_word': '{"1": "<oov>", "2": "f", "3": "c", "4": "e", "5": "h", "6": "g", "7": "b", "8": "d", "9": "a", "10": "i"}',
 'word_index': '{"<oov>": 1, "f": 2, "c": 3, "e": 4, "h": 5, "g": 6, "b": 7, "d": 8, "a": 9, "i": 10}'}

In [ ]:
train_alarmno_padseq.shape, train_alarmlv_padseq.shape, train_msgvec_padseq.shape, train_site1_padseq.shape, train_sysname1_padseq.shape

((891, 12, 1), (891, 12), (891, 12, 64), (891, 12), (891, 12))

In [ ]:
# 각 tokenizer들 json으로 저장하기
for name, data in zip(['alarmlv_tk', 'site1_tk', 'sysname1_tk'], [alarmlv_tokenizer, site1_tokenizer, sysname1_tokenizer]):
    with open(f'{tk_path}{name}.json', 'w', encoding='utf-8') as file:
        json.dump(data.to_json(), file)

## target data

In [ ]:
# one-hot encoding 해주기
y_train = to_categorical(train_seq[target], num_classes=len(label_dict))
y_valid = to_categorical(valid_seq[target], num_classes=len(label_dict))
y_train.shape, y_valid.shape

((891, 3), (223, 3))

# RNN modeling

In [ ]:
from tensorflow.keras.backend import clear_session

from tensorflow.keras.layers import Input, Dense, Embedding, Concatenate
from tensorflow.keras.layers import Bidirectional, GRU
from tensorflow.keras.layers import Flatten, MaxPool1D, Conv1D, Dropout, BatchNormalization

from tensorflow.keras.models import Model, load_model

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
clear_session()

# alarmno는 이미 수치화된 값이기에 Embedding layer가 필요 없다.
il_no = Input(shape=train_alarmno_padseq.shape[1:])
x_no = Bidirectional(layer=GRU(16, 'tanh'))(il_no)
x_no = Flatten()(x_no)

# alarmlevel
il_lv = Input(shape=train_alarmlv_padseq.shape[1:])
x_lv = Embedding(input_dim=alarmlevel_vocab_size, output_dim=128)(il_lv)
x_lv = Bidirectional(layer=GRU(128, 'tanh'))(x_lv)
x_lv = Flatten()(x_lv)

# alarmmsg_vector -> 단어 임베딩 모델이 이미 적용되어 Embedding layer가 필요 없다.
il_msg = Input(shape=train_msgvec_padseq.shape[1:])
x_msg = Bidirectional(layer=GRU(128, 'tanh'))(il_msg)
x_msg = Flatten()(x_msg)

# site_1
il_site1 = Input(shape=train_site1_padseq.shape[1:])
x_site1 = Embedding(input_dim=site1_vocab_size, output_dim=128)(il_site1)
x_site1 = Bidirectional(layer=GRU(128, 'tanh'))(x_site1)
x_site1 = Flatten()(x_site1)

# sysname_1
il_sysname1 = Input(shape=train_sysname1_padseq.shape[1:])
x_sysname1 = Embedding(input_dim=sysname1_vocab_size, output_dim=128)(il_sysname1)
x_sysname1 = Bidirectional(layer=GRU(128, 'tanh'))(x_sysname1)
x_sysname1 = Flatten()(x_sysname1)

x = Concatenate()([x_no, x_lv, x_msg, x_site1, x_sysname1])
x = Dropout(0.25)(x)
x = BatchNormalization()(x)
x = Dense(256, 'swish')(x)
x = Dropout(0.25)(x)
x = Dense(64, 'swish')(x)
x = Dropout(0.25)(x)
x = Dense(32, 'swish')(x)

ol = Dense(3, 'softmax')(x)


rnn_model = Model(inputs=[il_no, il_lv, il_msg, il_site1, il_sysname1], outputs=ol)

rnn_model.compile(loss='categorical_crossentropy',
              metrics=['accuracy'],
              optimizer='adam')
rnn_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 12)]         0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 12)]         0           []                               
                                                                                                  
 input_5 (InputLayer)           [(None, 12)]         0           []                               
                                                                                                  
 input_1 (InputLayer)           [(None, 12, 1)]      0           []                               
                                                                                              

In [ ]:
es = EarlyStopping(monitor='val_loss', min_delta=0, patience=20, verbose=1, restore_best_weights=1)
mcp = ModelCheckpoint(rnn_path,
                      monitor='val_loss',
                      min_delta=0,
                      patience=20,
                      save_best_only=True,
                      verbose=1)

In [ ]:
rnn_model.fit([train_alarmno_padseq, train_alarmlv_padseq, train_msgvec_padseq, train_site1_padseq, train_sysname1_padseq], y_train,
          validation_split=0.2, callbacks=[es, mcp],
          epochs=1000,
          verbose=1
          )

Epoch 1/1000
23/23 [==============================] - ETA: 0s - loss: 0.7133 - accuracy: 0.6770
Epoch 1: val_loss improved from inf to 0.96713, saving model to /content/drive/MyDrive/my_data/kt_network/model/q2model.h5
23/23 [==============================] - 31s 370ms/step - loss: 0.7133 - accuracy: 0.6770 - val_loss: 0.9671 - val_accuracy: 0.5251
Epoch 2/1000
23/23 [==============================] - ETA: 0s - loss: 0.4537 - accuracy: 0.7978
Epoch 2: val_loss improved from 0.96713 to 0.92595, saving model to /content/drive/MyDrive/my_data/kt_network/model/q2model.h5
23/23 [==============================] - 4s 150ms/step - loss: 0.4537 - accuracy: 0.7978 - val_loss: 0.9260 - val_accuracy: 0.6089
Epoch 3/1000
23/23 [==============================] - ETA: 0s - loss: 0.3538 - accuracy: 0.8694
Epoch 3: val_loss improved from 0.92595 to 0.89749, saving model to /content/drive/MyDrive/my_data/kt_network/model/q2model.h5
23/23 [==============================] - 3s 137ms/step - loss: 0.3538 - 

# Evaluation

In [ ]:
rnn_model = load_model(rnn_path)
rnn_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 12)]         0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 12)]         0           []                               
                                                                                                  
 input_5 (InputLayer)           [(None, 12)]         0           []                               
                                                                                                  
 input_1 (InputLayer)           [(None, 12, 1)]      0           []                               
                                                                                              

In [ ]:
rnn_model.evaluate(x=[valid_alarmno_padseq, valid_alarmlv_padseq, valid_msgvec_padseq, valid_site1_padseq, valid_sysname1_padseq], y=y_valid)

7/7 [==============================] - 4s 27ms/step - loss: 0.0429 - accuracy: 0.9910


[0.042859841138124466, 0.9910314083099365]